In [21]:
import pandas as pd
import os

In [22]:
Desktop = os.path.join(os.path.expanduser("~"), "Desktop")

filepath_root = os.path.join(Desktop, "LBP_Revision")
inclusion_root = os.path.join(filepath_root, "Inclusion_Codes") #inclusion code location
filtered_root = os.path.join(filepath_root, "Filtered_Files") #Filtered data location (Filtered data aggregated all diagnosis by patient)
unfiltered_root = os.path.join(filepath_root, "All_csv_files") #unfiltered data location
combined_root = os.path.join(filepath_root, "LBP_X_Files") #LBP + X filtered location
classify_root = os.path.join(filepath_root, "Classify_6") #6 groups location


os.path.exists(filepath_root)

path_inclusion_crit_LBP = os.path.join(inclusion_root,'inclusion_codes_LBP.xlsx')
path_inclusion_crit_DJD = os.path.join(inclusion_root,'inclusion_codes_DJD.xlsx')
path_inclusion_crit_fracture = os.path.join(inclusion_root,'inclusion_codes_fracture.xlsx')
path_inclusion_crit_other = os.path.join(inclusion_root,'inclusion_codes_other.xlsx')
path_inclusion_crit_scoliosis = os.path.join(inclusion_root,'inclusion_codes_scoliosis.xlsx')

In [23]:
def get_cpts_fromfile(filepath=''):
    df_cpt = pd.read_excel(filepath, dtype=str, sheet_name='CPT Codes')
    return df_cpt['CPT Code']       

def get_icds_fromfile(filepath=''):        
    df_icd = pd.read_excel(filepath,dtype=str, sheet_name='ICD Codes')
    return df_icd['ICD Code']

In [24]:
LBP_DJD_filename = 'LBP_DJD_patients'
LBP_fracture_filename = 'LBP_fracture_patients'
LBP_other_filename = 'LBP_other_patients'
LBP_scoliosis_filename = 'LBP_scoliosis_patients'
LBP_other_scoliosis_filename = 'LBP_other_scoliosis_patients'
LBP_DJD_data = pd.read_csv(combined_root + '/' + LBP_DJD_filename + '.csv', encoding = 'utf8',dtype=str, index_col=0)
LBP_fracture_data = pd.read_csv(combined_root + '/' + LBP_fracture_filename + '.csv', encoding = 'utf8',dtype=str, index_col=0)
LBP_other_data = pd.read_csv(combined_root + '/' + LBP_other_filename + '.csv', encoding = 'utf8',dtype=str, index_col=0)
LBP_scoliosis_data = pd.read_csv(combined_root + '/' + LBP_scoliosis_filename + '.csv', encoding = 'utf8',dtype=str, index_col=0)
LBP_other_scoliosis_data = pd.read_csv(combined_root + '/' + LBP_other_scoliosis_filename + '.csv', encoding = 'utf8',dtype=str, index_col=0)

LBP_all_data = pd.read_csv(unfiltered_root + '/' + 'LBP_all.csv', encoding = 'utf8',dtype=str)
DJD_all_data = pd.read_csv(unfiltered_root + '/' + 'DJD_all.csv', encoding = 'utf8',dtype=str)
fracture_all_data = pd.read_csv(unfiltered_root + '/' + 'fracture_all.csv', encoding = 'utf8',dtype=str)
other_all_data = pd.read_csv(unfiltered_root + '/' + 'other_all.csv', encoding = 'utf8',dtype=str)
scoliosis_all_data = pd.read_csv(unfiltered_root + '/' + 'scoliosis_all.csv', encoding = 'utf8',dtype=str)

In [25]:
LBP_filtered_data = pd.read_csv(filtered_root + '/' + 'LBP_patients.csv', encoding = 'utf8',dtype=str, index_col=0)
DJD_filtered_data = pd.read_csv(filtered_root + '/' + 'DJD_patients.csv', encoding = 'utf8',dtype=str, index_col=0)
fracture_filtered_data = pd.read_csv(filtered_root + '/' + 'fracture_patients.csv', encoding = 'utf8',dtype=str, index_col=0)
other_filtered_data = pd.read_csv(filtered_root + '/' + 'other_patients.csv', encoding = 'utf8',dtype=str, index_col=0)
scoliosis_filtered_data = pd.read_csv(filtered_root + '/' + 'scoliosis_patients.csv', encoding = 'utf8',dtype=str, index_col=0)

In [26]:
control_all_data = pd.read_csv(unfiltered_root + '/' + 'control_all.csv', encoding = 'utf8',dtype=str)
control_diagcode = control_all_data[['EMPI','Age','Race1','BMI','Code.prc','Code.dia','Helper']]
control_diagcode['Diagnosis_combined'] = str(0)
print(control_diagcode.keys())

control_diagcode_reference = control_diagcode

Index(['EMPI', 'Age', 'Race1', 'BMI', 'Code.prc', 'Code.dia', 'Helper',
       'Diagnosis_combined'],
      dtype='object')


/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_16826/4055427511.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_diagcode['Diagnosis_combined'] = str(0)


In [27]:
control_diagcode.loc[0,'Diagnosis_combined'] = control_diagcode.loc[0,'Code.dia']
for i in range(1,len(control_diagcode)):
    if (control_diagcode.loc[i,'EMPI'] == control_diagcode.loc[i-1,'EMPI']):
        if control_diagcode.loc[i,'Code.dia'] != control_diagcode.loc[i-1,'Code.dia']:
            control_diagcode.loc[i,'Diagnosis_combined'] = control_diagcode.loc[i,'Code.dia'] + ', ' + str(control_diagcode.loc[i-1,'Diagnosis_combined'])
            continue;
        else: 
            control_diagcode.loc[i,'Diagnosis_combined'] = control_diagcode.loc[i-1,'Diagnosis_combined']
            continue;
    else: 
        control_diagcode.loc[i,'Diagnosis_combined'] = control_diagcode.loc[i,'Code.dia']
        continue;

In [28]:
control_filtered = control_diagcode[control_diagcode['Helper'].notna()]
control_filtered_save = control_filtered

In [29]:
control_filtered.head(20)
print(len(control_filtered))

1235


In [30]:
#CPT
incl_cpts_LBP = get_cpts_fromfile(path_inclusion_crit_LBP)
incl_cpts_text_LBP = '|'.join(incl_cpts_LBP.str.strip())

incl_cpts_DJD = get_cpts_fromfile(path_inclusion_crit_DJD)
incl_cpts_text_DJD = '|'.join(incl_cpts_DJD.str.strip())

incl_cpts_fracture = get_cpts_fromfile(path_inclusion_crit_fracture)
incl_cpts_text_fracture = '|'.join(incl_cpts_fracture.str.strip())

incl_cpts_other = get_cpts_fromfile(path_inclusion_crit_other)
incl_cpts_text_other = '|'.join(incl_cpts_other.str.strip())

incl_cpts_scoliosis = get_cpts_fromfile(path_inclusion_crit_scoliosis)
incl_cpts_text_scoliosis = '|'.join(incl_cpts_scoliosis.str.strip())

#ICD
incl_icds_LBP = get_icds_fromfile(path_inclusion_crit_LBP)
incl_icds_text_LBP = '|'.join(incl_icds_LBP.str.strip())

incl_icds_DJD = get_icds_fromfile(path_inclusion_crit_DJD)
incl_icds_text_DJD = '|'.join(incl_icds_DJD.str.strip())

incl_icds_fracture = get_icds_fromfile(path_inclusion_crit_fracture)
incl_icds_text_fracture = '|'.join(incl_icds_fracture.str.strip())

incl_icds_other = get_icds_fromfile(path_inclusion_crit_other)
incl_icds_text_other = '|'.join(incl_icds_other.str.strip())

incl_icds_scoliosis = get_icds_fromfile(path_inclusion_crit_scoliosis)
incl_icds_text_scoliosis = '|'.join(incl_icds_scoliosis.str.strip())


In [31]:
#Case group LBP
CG_LBP = LBP_filtered_data

#Case group 1
CG1_fracture = LBP_DJD_data.loc[~((LBP_DJD_data.EMPI.isin(fracture_filtered_data['EMPI']))),:]
CG1_other = CG1_fracture.loc[~((CG1_fracture.EMPI.isin(other_filtered_data['EMPI']))),:]
CG1_scoliosis = CG1_other.loc[~((CG1_other.EMPI.isin(scoliosis_filtered_data['EMPI']))),:]
CG1 = CG1_scoliosis

#Case group 2
CG2_DJD = LBP_fracture_data.loc[~((LBP_fracture_data.EMPI.isin(DJD_filtered_data['EMPI']))),:]
CG2_other = CG2_DJD.loc[~((CG2_DJD.EMPI.isin(other_filtered_data['EMPI']))),:]
CG2_scoliosis = CG2_other.loc[~((CG2_other.EMPI.isin(scoliosis_filtered_data['EMPI']))),:]
CG2 = CG2_scoliosis

#Case group 3
CG3_DJD = LBP_other_scoliosis_data.loc[~((LBP_other_scoliosis_data.EMPI.isin(DJD_filtered_data['EMPI']))),:]
CG3_fracture = CG3_DJD.loc[~((CG3_DJD.EMPI.isin(fracture_filtered_data['EMPI']))),:]
CG3 = CG3_fracture

#Case group 4
CG4_DJD = LBP_filtered_data.loc[~((LBP_filtered_data.EMPI.isin(DJD_filtered_data['EMPI']))),:]
CG4_fracture = CG4_DJD.loc[~((CG4_DJD.EMPI.isin(fracture_filtered_data['EMPI']))),:]
CG4_other = CG4_fracture.loc[~((CG4_fracture.EMPI.isin(other_filtered_data['EMPI']))),:]
CG4_scoliosis = CG4_other.loc[~((CG4_other.EMPI.isin(scoliosis_filtered_data['EMPI']))),:]
CG4 = CG4_scoliosis

#Case group control
CG_control = control_all_data


In [32]:
#Compare two different method
CG1_other = CG1_fracture.loc[~((CG1_fracture.EMPI.isin(other_filtered_data['EMPI']))),:]
CG1_other2 = LBP_DJD_data.loc[~((LBP_DJD_data.EMPI.isin(fracture_filtered_data['EMPI']))&(LBP_DJD_data.EMPI.isin(other_filtered_data['EMPI']))),:]
CG1_other.head()
CG1_other2.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,DJD?,Both
0,100000152,81,White,27.02,72100,G89.29,useful,"G89.29, M51.86",LBP,DJD,Both
7,100003506,96,Declined,20.06,72100,M47.816,useful,"M47.816, M54.9",LBP,DJD,Both
8,100004276,92,Other,21.15,72100,M54.9,useful,"M54.9, M47.816, G89.29",LBP,DJD,Both
13,100006348,80,White,27,72040,G89.29,useful,"G89.29, M47.12",LBP,DJD,Both
19,100008727,79,White,NaN,72100,M51.36,useful,"M51.36, G89.21",LBP,DJD,Both


In [33]:
print(len(CG1),len(CG2),len(CG3),len(CG4))
print(len(LBP_filtered_data))
#print(len())

766 540 188 531
5220


In [34]:
CG_LBP_all = LBP_all_data.loc[((LBP_all_data.EMPI.isin(CG_LBP['EMPI']))),:]
CG1_all = LBP_all_data.loc[((LBP_all_data.EMPI.isin(CG1['EMPI']))),:]
CG2_all = LBP_all_data.loc[((LBP_all_data.EMPI.isin(CG2['EMPI']))),:]
CG3_all = LBP_all_data.loc[((LBP_all_data.EMPI.isin(CG3['EMPI']))),:]
CG4_all = LBP_all_data.loc[((LBP_all_data.EMPI.isin(CG4['EMPI']))),:]
CG_control_all = control_all_data.loc[((control_all_data.EMPI.isin(CG_control['EMPI']))),:]

In [35]:
CG4_all.head()

,EMPI,MGH_MRN,Date.prc,Date.dia,Code_Type.prc,Code.prc,Code_Type.dia,Code.dia,Diagnosis_Name,Procedure_Name,...,Insurance_2,Insurance_3,Phone_Code,Physical_Date,BMI,Height,Weight,Smoking,Alcohol_Use,Helper
34,100022164,3136084,7/5/16,7/12/16,CPT,72100,ICD10,G89.29,Other chronic pain,"Radiologic examination, spine, lumbosacral; tw...",...,MASSHEALTH,MEDICARE,617,7/4/16,NaN,NaN,NaN,Smokeless Tobacco Use-Never Used,Alcohol User-No,NaN
35,100022164,3136084,7/5/16,7/12/16,CPT,72100,ICD10,G89.29,Other chronic pain,"Radiologic examination, spine, lumbosacral; tw...",...,MASSHEALTH,MEDICARE,617,7/4/16,NaN,NaN,NaN,Smokeless Tobacco Use-Never Used,Alcohol User-No,NaN
36,100022164,3136084,7/5/16,7/12/16,CPT,72100,ICD10,G89.29,Other chronic pain,"Radiologic examination, spine, lumbosacral; tw...",...,MASSHEALTH,MEDICARE,617,7/4/16,NaN,NaN,NaN,Smokeless Tobacco Use-Never Used,Alcohol User-No,useful
89,100061862,764138,12/16/16,12/16/16,CPT,72040,ICD10,G89.29,Other chronic pain,"Radiologic examination, spine, cervical; two o...",...,TUFTS HEALTH PLAN,MEDICARE,781,12/14/16,28.57,167.64,80.285784,Tobacco User-Never,Alcohol User-Yes,NaN
90,100061862,764138,12/16/16,12/16/16,CPT,72040,ICD10,G89.29,Other chronic pain,"Radiologic examination, spine, cervical; two o...",...,TUFTS HEALTH PLAN,MEDICARE,781,12/14/16,28.57,167.64,80.285784,Tobacco User-Never,Alcohol User-Yes,useful


In [36]:
CG_LBP_all = CG_LBP_all[CG_LBP_all['Helper'].notna()]
CG1_all = CG1_all[CG1_all['Helper'].notna()]
CG2_all = CG2_all[CG2_all['Helper'].notna()]
CG3_all = CG3_all[CG3_all['Helper'].notna()]
CG4_all = CG4_all[CG4_all['Helper'].notna()]
CG_control_all = CG_control_all[CG_control_all['Helper'].notna()]

In [37]:
CG4_all.head()

,EMPI,MGH_MRN,Date.prc,Date.dia,Code_Type.prc,Code.prc,Code_Type.dia,Code.dia,Diagnosis_Name,Procedure_Name,...,Insurance_2,Insurance_3,Phone_Code,Physical_Date,BMI,Height,Weight,Smoking,Alcohol_Use,Helper
36,100022164,3136084,7/5/16,7/12/16,CPT,72100,ICD10,G89.29,Other chronic pain,"Radiologic examination, spine, lumbosacral; tw...",...,MASSHEALTH,MEDICARE,617,7/4/16,NaN,NaN,NaN,Smokeless Tobacco Use-Never Used,Alcohol User-No,useful
90,100061862,764138,12/16/16,12/16/16,CPT,72040,ICD10,G89.29,Other chronic pain,"Radiologic examination, spine, cervical; two o...",...,TUFTS HEALTH PLAN,MEDICARE,781,12/14/16,28.57,167.64,80.285784,Tobacco User-Never,Alcohol User-Yes,useful
122,100085177,1922183,1/7/16,2/10/16,CPT,74000,ICD10,G89.29,Other chronic pain,"Radiologic examination, abdomen; single antero...",...,MASSHEALTH,NaN,617,12/16/15,56.62,NaN,184.158352,Smokeless Tobacco Use-Unknown,Alcohol User-Not Asked,useful
133,100091176,1074012,7/14/16,10/7/16,CPT,74000,ICD10,G89.29,Other chronic pain,"Radiologic examination, abdomen; single antero...",...,BLUE CROSS BLUE SHIELD,MEDICARE,978,7/14/16,32.52,NaN,78.017824,Smokeless Tobacco Use-Unknown,Alcohol User-No,useful
161,100112336,1243772,11/21/16,11/21/16,CPT,72100,ICD10,M54.9,"Dorsalgia, unspecified","Radiologic examination, spine, lumbosacral; tw...",...,BLUE CROSS BLUE SHIELD,MEDICARE,508,10/7/16,24.96,NaN,71.213944,Tobacco User-Quit,Alcohol User-No,useful


In [38]:
CG_LBP_all = CG_LBP_all.drop(['diagnosis_list','Diagnosis_list_combined'], axis=1)
CG1_all = CG1_all.drop(['diagnosis_list','Diagnosis_list_combined'], axis=1)
CG2_all = CG2_all.drop(['diagnosis_list','Diagnosis_list_combined'], axis=1)
CG3_all = CG3_all.drop(['diagnosis_list','Diagnosis_list_combined'], axis=1)
CG4_all = CG4_all.drop(['diagnosis_list','Diagnosis_list_combined'], axis=1)
CG_control_all = CG_control_all.drop(['diagnosis_list','Diagnosis_list_combined'], axis=1)

In [39]:
CG_LBP_all_csv_path = os.path.join(classify_root,'CG_LBP.csv')
CG_LBP_all.to_csv(CG_LBP_all_csv_path)

CG1_all_csv_path = os.path.join(classify_root,'CG1.csv')
CG1_all.to_csv(CG1_all_csv_path)

CG2_all_csv_path = os.path.join(classify_root,'CG2.csv')
CG2_all.to_csv(CG2_all_csv_path)

CG3_all_csv_path = os.path.join(classify_root,'CG3.csv')
CG3_all.to_csv(CG3_all_csv_path)

CG4_all_csv_path = os.path.join(classify_root,'CG4.csv')
CG4_all.to_csv(CG4_all_csv_path)

CG_control_all_csv_path = os.path.join(classify_root,'CG_control.csv')
CG_control_all.to_csv(CG_control_all_csv_path)

In [40]:
print(len(CG_LBP_all), len(CG1_all), len(CG2_all), len(CG3_all), len(CG4_all), len(CG_control_all))

5220 896 569 209 531 1235
